### Build a list with dictionaries including below data of each unit
- {
- 'musicname':'original filename',
- 'staff_id':int,
- 'start':int,
- 'end':int,
- 'tempo':int,
- 'beat':'1/4,0; 1/2,1; 1/8,1; 1/8,0' (假設為4/4拍，左為一單位的拍子，順序為：四分休止符、二分音符、八分音符、八分休止符，1/4 + 1/2 + 1/8 + 1/8 = 1)
- }

In [2]:
from bs4 import BeautifulSoup as bs4
from decimal import *
import os

#Determine tempo, start measure & end measure
#for each unit(4 measures(3, 2 or 1 measure for last unit)).
def getTempoMeasure(filePath, mainList):
    with open(filePath, 'r') as f:
        mscx = bs4(f.read(), 'xml')
    
    #Get the filename.
    
    #Select measures of staff#1 because tempos only appear in staff#1.
    staff1Measures = mscx.select('Score > Staff:nth-of-type(1) > Measure')
    
    #Get the 1st tempo.
    tempo = staff1Measures[0].select('Tempo > text')[0].text.rsplit(' ', 1)[-1]
    
    #Set tempos & measure numbers to the list.
    tmp = []
    for i, m in enumerate(staff1Measures[:-1], 1):
        for t in m.select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if i % 4 == 0:
            if tmp:
                tempo = max(tmp)
            mainList.append(genDataDic(tempo, i-3, i))
            tmp[:] = []
    
    #Different situations of the last measure handled by "for else".
    else:
        i += 1
        for t in staff1Measures[-1].select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if tmp:
            tempo = max(tmp)    
        if i % 4 == 1: 
            mainList.append(genDataDic(tempo, i, i))
        elif i % 4 == 2:
            mainList.append(genDataDic(tempo, i-1, i))
        elif i % 4 == 3:
            mainList.append(genDataDic(tempo, i-2, i))
        else:
            mainList.append(genDataDic(tempo, i-3, i))

    for e in mainList:
        print e


def genDataDic(tempo, start, end):
    return {'tempo':tempo, 'start':start, 'end':end}

In [5]:
#The mainList for storing data dictionaries for each unit
mainList = []

#Determine tempo, start measure & end measure
#for each unit(4 measures(3, 2 or 1 measure for last unit)).
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    filePath = dirPath + filename
    getTempoMeasure(filePath, mainList)

{'start': 1, 'end': 4, 'tempo': 74}
{'start': 5, 'end': 8, 'tempo': 74}
{'start': 9, 'end': 12, 'tempo': 70}
{'start': 13, 'end': 16, 'tempo': 70}
{'start': 17, 'end': 20, 'tempo': 70}
{'start': 21, 'end': 24, 'tempo': 70}
{'start': 25, 'end': 28, 'tempo': 70}
{'start': 29, 'end': 32, 'tempo': 70}
{'start': 33, 'end': 36, 'tempo': 70}
{'start': 37, 'end': 40, 'tempo': 70}
{'start': 41, 'end': 44, 'tempo': 70}
{'start': 45, 'end': 48, 'tempo': 70}
{'start': 49, 'end': 52, 'tempo': 70}
{'start': 53, 'end': 56, 'tempo': 70}
{'start': 57, 'end': 60, 'tempo': 70}


In [44]:
#The beat dictionary.
beatDic = {'measure':None,
           'whole':1,
           'half':Decimal(1)/2,
           'quarter':Decimal(1)/4,
           'eighth':Decimal(1)/8,
           '16th':Decimal(1)/16,
           '32nd':Decimal(1)/32,
           '64th':Decimal(1)/64,
           '128th':Decimal(1)/128,
           '256th':Decimal(1)/256
          }

In [45]:
#After having tempo, start measure & end measure
#also put beats into the list.
#Note: About <tick><track> tags indicating at least
#1 additional melody in a staff, omit them at first.
def getBeat(filePath, mmList, mainDic):
    with open(filePath, 'r') as f:
        mscx = bs4(f.read(), 'xml')
    
    #Determine if there is more than 1 type of Time Signature.
    #If there is more than 1, detection of Time Signature
    #is required at each measure.
    staff1TimeSig = mscx.select('Score > Staff:nth-of-type(1) TimeSig')
    if len(staff1TimeSig) == 1:
        only1 = True
    else:
        only1 = False
    
    sigN = int(staff1TimeSig[0].find('sigN').text)
    sigD = int(staff1TimeSig[0].find('sigD').text)
    beatDic['measure'] = Decimal(sigN)/sigD
        
    for i, m in enumerate(mscx.select('Score > \
                              Staff:nth-of-type(1) > \
                              Measure'), 1):
        for e in m.select('Chord,  Rest'):
            print e.prettify()
            break
    

filePath = 'C:/Users/BigData/Desktop/Everybody Wants To Rule The World.mscx'
    
#The list for the staffs which play main melodies
mmList = [1, 2, 3]

mainDic = getBeat(filePath, mmList, mainDic)

0.75
<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Chord>
 <durationType>
  eighth
 </durationType>
 <Note>
  <pitch>
   86
  </pitch>
  <tpc>
   16
  </tpc>
  <velocity>
   80
  </velocity>
  <veloType>
   user
  </veloType>
 </Note>
</Chord>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </durationType>
 <duration n="8" z="6"/>
</Rest>

<Rest>
 <durationType>
  measure
 </duration